In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df = pd.read_csv("ncov-or-cov-19-or-covid-o-all-story-urls-20200628121924.csv")
df.head()

In [ ]:
# Selecting only rows where files exist

files = []
for file in os.listdir('Mediacloud/'):
    files.append(file[:-4])

df = df[df['stories_id'].isin(files)]
df['Text'] = None

In [ ]:
#Reading files and adding text to the dataframe

for file in os.listdir('Mediacloud/'):
    with open('Mediacloud/' + file, 'r') as text:
        data = text.read().replace('\n', '')
        df.loc[df['stories_id'] == int(file[:-4]), 'Text'] = data 

In [ ]:
#Calculating text length and briefly describing it

df['text_len'] = df['Text'].apply(lambda x: len(str(x).split()) if x else np.nan)
print(df['text_len'].describe())
df.hist(column='text_len', bins=15, grid=False, figsize=(12,8), color='#86bf91', zorder=2, rwidth=0.9)

In [ ]:
# Selecting those rows where more than 100 words are present in the text

df = df[df['text_len'] > 100].reset_index()
df.shape

In [ ]:
#Processing the text and returning tokens

def text_processing(input_str):
    input_lower = input_str.lower()
    input_punctutation = input_lower.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).replace(' '*4, ' ').replace(' '*3, ' ').replace(' '*2, ' ').strip()
    input_tokens = nltk.word_tokenize(input_punctutation)
    input_stopwords = [i for i in input_tokens if not i in stop_words]
    lemmatizer=WordNetLemmatizer()
    input_lemmatized = [lemmatizer.lemmatize(word) for word in input_stopwords]
    return (input_lemmatized)
    
df['tokens'] = df['Text'].apply(lambda x: text_processing(x))

In [ ]:
#count the frequency of words and return freq of corona-related terms

def wordListToFreqDict(wordlist, terms):
    wordfreq = [wordlist.count(p) for p in wordlist]
    res = dict(list(zip(wordlist,wordfreq)))
    return [(term, res[term]) for term in terms if term in res.keys()]

with open('corona_terms.txt', 'r') as corona_terms:
    terms = corona_terms.read().replace('\n', ' ').split(' ')

df['corona_terms'] = df['tokens'].apply(lambda x: wordListToFreqDict(x, terms))
df['corona_freq'] = df['corona_terms'].apply(lambda x: sum([item[1] for item in x]))

In [ ]:
df['corona_freq'].describe()

In [ ]:
df_corona = df[df['corona_freq'] >3]
df_corona.shape

In [ ]:
df['Text'][3499]